<a href="https://colab.research.google.com/github/Ignaciovf/timeseries_lab/blob/main/LSTM_PM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
print('Tensorflow Version: ',tf.__version__) # Revisamos la versión de TF


Tensorflow Version:  2.4.1


Subimos el dataset que tengamos a Colab, hay que tener cuidado que se borrará en cuanto se reinicie el kernel

In [18]:
df=pd.read_csv('/content/VDE_daily_madrid_pollution.csv')

In [19]:
# Creamos una función para normalizar los dataframes que tengamos
# La normalización la vamos a realizar con los datos del train y no con los del test para evitar obtener resultados no generalizables. 
def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [47]:
# feature_keys=df.columns
# Definimos qué columnas vamos a utilizar
selected_columns = ['PM_CENTRO','HUMIDITY', 'PRECIPITAITON']
df.index = df['Date']
df=df[selected_columns]

In [48]:
# Definimos el número de datos que vamos a incluír en el train y test
# Elegimos 60 días para el train
test_size=60
train_size=df.shape[0]-test_size
# definimos nuestro start y end date
train_start_date = df.head(1).index.values[0]
test_start_date=df.iloc[train_size:train_size+1].index.values[0]
test_end_date = df.tail(1).index.values[0]
print('Train empieza el ', train_start_date)
print('Test empieza el ', test_start_date)
print('Test termina el ',test_end_date)

Train empieza el  2010-01-01
Test empieza el  2015-11-02
Test termina el  2015-12-31


In [49]:
# Vamos a normalizar los valores de los valores que vamos a seleccionar para el dataframe
df = normalize(df.values, train_size)
df = pd.DataFrame(regresors)
df.head()

,PM_CENTRO,HUMIDITY,PRECIPITAITON
Date,,,
2010-01-01,4.859812,3.649575,0.000000
2010-01-02,4.972125,4.355907,0.000000
2010-01-03,4.361505,4.476389,2.501436
2010-01-04,3.377303,3.833160,0.000000
2010-01-05,3.773718,3.738661,0.000000


Vamos a crear los datos que necesitamos para entrenar el modelo 

In [53]:
df.head()

,PM_CENTRO,HUMIDITY,PRECIPITAITON
Date,,,
2010-01-01,4.859812,3.649575,0.000000
2010-01-02,4.972125,4.355907,0.000000
2010-01-03,4.361505,4.476389,2.501436
2010-01-04,3.377303,3.833160,0.000000
2010-01-05,3.773718,3.738661,0.000000


In [68]:
df.index = pd.to_datetime(df.index)
train_data = df.loc[pd.to_datetime(train_start_date) : pd.to_datetime(test_start_date)+timedelta(days=-1)]
val_data = df.loc[pd.to_datetime(train_start_date):]


# x_train = train_data[[i for i in range(len(df.columns))]].values
# y_train = df.iloc[:train_size][[1]]

# print(x_train)

In [67]:
pd.to_datetime(test_start_date)

Timestamp('2015-11-03 00:00:00')

In [63]:
help(timedelta)

Help on class timedelta in module datetime:

class timedelta(builtins.object)
 |  Difference between two datetime values.
 |  
 |  timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
 |  
 |  All arguments are optional and default to 0.
 |  Arguments may be integers or floats, and may be positive or negative.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, v